In [2]:
import os
import sys
os.environ["OMP_NUM_THREADS"] = "11"
os.environ["OPENBLAS_NUM_THREADS"] = "8" # export OPENBLAS_NUM_THREADS=4 
os.environ["MKL_NUM_THREADS"] = "11" # export MKL_NUM_THREADS=6
os.environ["VECLIB_MAXIMUM_THREADS"] = "8" # export VECLIB_MAXIMUM_THREADS=4
os.environ["NUMEXPR_NUM_THREADS"] = "11" # export NUMEXPR_NUM_THREADS=6
os.environ["NUMBA_CACHE_DIR"]='/tmp/numba_cache'
import numpy as np
import pandas as pd
import scipy 
import scipy.io as sio
import scipy.sparse as sps
import h5py
from types import SimpleNamespace

import tensorflow as tf
import matplotlib.pyplot as plt
import scanpy as sc

from os.path import join

os.environ["CUDA_VISIBLE_DEVICES"] = '1'
# physical_devices = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
# except:
#     # Invalid device or cannot modify virtual devices once initialized.
#     pass

In [3]:
import scipy
from sklearn.metrics import roc_auc_score
def pearson_mat(X, Y):
    X = (X - X.mean(axis=0))
    X /= (scipy.linalg.norm(X, axis=0, ord=2) + 1e-12)
    Y = (Y - Y.mean(axis=0))
    Y /= (scipy.linalg.norm(Y, axis=0, ord=2) + 1e-12)
    return (X * Y).sum(axis=0)

def eval_pearRmse_AlongGene(X, Y):
    pear = pearson_mat(X, Y)
    rmse = np.sqrt(np.mean((X-Y)**2, axis=0))
    return pear, rmse

def eval_spear_AlongGene(X, Y):
    spears = []
    for gi in range(X.shape[1]):
        spears.append(scipy.stats.spearmanr(X[:, gi], Y[:, gi])[0])
    return spears

def eval_aucRmse_AlongPeak(X, Y):
    aucs, rmses = [], []
    for pi in range(X.shape[1]):
        aucs.append(roc_auc_score(X[:, pi], Y[:, pi]))
        rmses.append(
            np.sqrt(np.mean((X[:, pi] - Y[:, pi])**2))
        )
    return aucs, rmses

def eval_imputation_flatten(x, y):
    pearson_r, pearson_p = scipy.stats.pearsonr(x, y)
    print(f"Found pearson's correlation/p of {pearson_r:.4f}/{pearson_p:.4g}")
    spearman_corr, spearman_p = scipy.stats.spearmanr(x, y)
    print(f"Found spearman's collelation/p of {spearman_corr:.4f}/{spearman_p:.4g}")
    rmse = np.sqrt(np.mean((x - y)**2))
    print(f"Found rmse {rmse:.4f}")
    return pearson_r, spearman_corr, rmse

# Load data

In [4]:
data_dir = "/home/sda1/yanxh/data/Seurat_demo_data/pbmc_multiome"

# print('Reading `mtx` files...')
X = sps.csr_matrix(sio.mmread(join(data_dir, 'rna_mat_norm.mtx')).T)
Y = sps.csr_matrix(sio.mmread(join(data_dir, 'atac_mat_norm.mtx')).T)
rna_names = pd.read_csv(join(data_dir, 'gene_names.csv'))['x'].to_numpy()
atac_names = pd.read_csv(join(data_dir, 'atac_names.csv'))['x'].to_numpy()

cell_names = pd.read_csv(join(data_dir, 'cell_names.csv'))['x'].to_numpy()
meta_data = pd.read_csv(join(data_dir, 'metadata.csv'), index_col=0)

train_idx = pd.read_csv(join(data_dir, 'train_idx.csv'))['0'].to_numpy()
test_idx  = pd.read_csv(join(data_dir, 'test_idx.csv'))['0'].to_numpy()

# select hvg and hvp
ad_rna = sc.AnnData(X, obs=meta_data.loc[cell_names])
sc.pp.highly_variable_genes(ad_rna, n_top_genes=5000)
hvg_idx = np.where(ad_rna.var.highly_variable)[0]

# pick peak startwith chr1-23
valid_atac_idx = [
    _ for _ in range(len(atac_names)) 
    if atac_names[_].startswith('chr') and 
    not atac_names[_].startswith('chrX-') and 
    not atac_names[_].startswith('chrY-')
]
valid_atac_names = atac_names[valid_atac_idx]
Y = Y[:, valid_atac_idx]

hvp_idx = np.argsort(Y.sum(axis=0).A1)[-20000:]
hvp_names = valid_atac_names[hvp_idx]
sort_idx = np.argsort(hvp_names)
sorted_hvp_idx, sorted_hvp_names = hvp_idx[sort_idx], hvp_names[sort_idx]

/tmp/ipykernel_18534/2863803315.py:16: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  ad_rna = sc.AnnData(X, obs=meta_data.loc[cell_names])


In [5]:
mult_X = X[train_idx][:, hvg_idx].A
mult_Y = Y[train_idx][:, sorted_hvp_idx].A
single_X  = X[test_idx][:, hvg_idx].A
single_Y  = Y[test_idx][:, sorted_hvp_idx].A

# binarize atac data
mult_Y[mult_Y>0.] = 1.
single_Y[single_Y>0.] = 1.

n_mult, n_rna, n_atac = mult_X.shape[0], single_X.shape[0], single_Y.shape[0]
m_rna, m_atac = mult_X.shape[1], mult_Y.shape[1]

# Count peaks in each chromosome (assuming they are ordered)
chr_set, chr_n_chunk = np.unique([_.split('-')[0] for _ in sorted_hvp_names], return_counts=True)

In [6]:
# for imputation
mult_data = np.hstack([mult_X, mult_Y])

In [7]:
dim_input_arr = [m_rna, m_atac]
config = {
    'dim_input_arr': dim_input_arr,
    'dimensions':[256], 
    'dim_latent':32,
    'dim_block': np.append([m_rna], chr_n_chunk), 
    'dist_block':['NB'] + ['Bernoulli' for _ in chr_n_chunk], 
    'dim_block_enc':np.array([256] + [16 for _ in chr_n_chunk]),
    'dim_block_dec':np.array([256] + [16 for _ in chr_n_chunk]),
    'dim_block_embed':np.array([16] + [1 for _ in range(len(chr_n_chunk))])*2,
    
    'block_names':np.array(['rna'] + ['atac' for _ in range(len(chr_n_chunk))]),
    'uni_block_names':np.array(['rna','atac']),
    
    'beta_kl':1.,
    'beta_unobs':2./3.,
    'beta_modal':np.array([0.8, 0.2]),  # not very sure
    'beta_reverse':0.5,

    "p_feat" : 0.2,
    "p_modal" : np.ones(2)/2,
    
}
config = SimpleNamespace(**config)
n_samples = 50

In [8]:
from functools import partial
from scVAEIT.VAEIT import scVAEIT
# model = scVAEIT(config, mosaic_data, masks, batch_ids)  # for integration
model = scVAEIT(config, mult_data)

2023-10-01 19:02:46.513898: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-01 19:02:46.933649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22306 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [9]:
if False:
    model.train(
        valid=False, num_epoch=300, batch_size=512, save_every_epoch=50,
        verbose=True, checkpoint_dir='./checkpoint/pbmc-mult-impute'
    )

# load the model and ensure it is loaded successfully
checkpoint = tf.train.Checkpoint(net=model.vae)
epoch = 10
status = checkpoint.restore('checkpoint/pbmc-mult-impute/ckpt-{}'.format(epoch))

# one-step forward?
model.vae(tf.zeros((1,np.sum(model.vae.config.dim_input_arr))),
          tf.zeros((1,np.sum(model.vae.config.dim_input_arr))),
          tf.zeros((1,np.sum(model.batches.shape[1]))), 
          pre_train=True, L=1, training=False)
print(status)

2023-10-01 19:02:51.083789: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [10]:
dataset_test = tf.data.Dataset.from_tensor_slices((
    np.hstack([single_X, single_Y]),
    model.cat_enc.transform(np.zeros((test_idx.size, 1))).toarray().astype(np.float32),
    np.zeros(test_idx.size).astype(np.int32)
)).batch(512).prefetch(tf.data.experimental.AUTOTUNE)

mask_rna = np.zeros((1, m_rna+m_atac), dtype=np.float32)
mask_rna[:,:m_rna] = -1.
recon = model.vae.get_recon(dataset_test, mask_rna)
X_hat = recon[:, :m_rna]

mask_atac = np.zeros((1, m_rna+m_atac), dtype=np.float32)
mask_atac[:, m_rna:] = -1.
recon = model.vae.get_recon(dataset_test, mask_atac)
Y_hat = recon[:, m_rna:]

In [10]:
# batch2: rna->atac
x, y = single_Y, Y_hat

auc = roc_auc_score(x.flatten(), y.flatten())
# rmse = np.sqrt(((x.flatten()-y.flatten())**2).mean())

auc_along_peak, rmse_along_peak = eval_aucRmse_AlongPeak(x, y)
auc, np.mean(auc_along_peak) # np.mean(rmse_along_peak)

(0.7739375580477281, 0.6450076955902503)

In [11]:
# batch3: atac->rna
pear1, spear1, rmse = eval_imputation_flatten(single_X.flatten(), X_hat.flatten())
pear_along_gene, rmse_along_gene = eval_pearRmse_AlongGene(single_X, X_hat)
spear_along_gene = eval_spear_AlongGene(single_X, X_hat)

np.mean(pear_along_gene), np.mean(spear_along_gene)#, np.mean(rmse_along_gene)

Found pearson's correlation/p of 0.7171/0
Found spearman's collelation/p of 0.3409/0
Found rmse 0.2935


/home/yanxh/anaconda3/envs/tf2-py38/lib/python3.8/site-packages/scipy/stats/_stats_py.py:4878: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


(0.14195727723678903, nan)

In [12]:
save_dir = '/home/yanxh/gitrepo/multi-omics-matching/Visualization/outputs/imputation'

gene_metcs = np.vstack([pear_along_gene, spear_along_gene]).T
_df1 = pd.DataFrame(gene_metcs, index=rna_names[hvg_idx], columns=['pear', 'spear'])
_df1.to_csv(join(save_dir, 'scVAEIT_pbmc-mult_along_gene.csv'))

peak_metcs = auc_along_peak
_df2 = pd.DataFrame(peak_metcs, index=sorted_hvp_names, columns=['auc'])
_df2.to_csv(join(save_dir, 'scVAEIT_pbmc-mult_along_peak.csv'))

In [14]:
np.save('./checkpoint/pbmc-mult-impute/X_test_imputed.npy', X_hat)
np.save('./checkpoint/pbmc-mult-impute/Y_test_imputed.npy', Y_hat)